Dynamics of offset diff drive vehicle

Equations are from the paper written by Masayoshi Wada etal.

https://www.jstage.jst.go.jp/article/jrsj1983/18/8/18_8_1166/_pdf

This notebook is written by Yosuke Matsusaka

# Preparation

We use sympy to transform the equation:

In [1]:
!sudo pip install -U sympy

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
Requirement already up-to-date: sympy in /usr/local/lib/python2.7/dist-packages (1.5.1)


In [2]:
from sympy import *

In [3]:
init_printing()

# Symbols

Following symbols are used to describe internal state of the vehicle:

In [4]:
(r, s, W, theta) = symbols(r"r s W \theta_s")

$r$ : radius of each wheels

$W$: wheel separation

$s$ : wheel offset

$\theta_s$ : current angle of the steering joint (described later)

Joint space parameters:

In [5]:
(wr, wl, ws) = symbols('w_r w_l w_s')
uv = Matrix([wr, wl, ws])

$w_r$ : angular velocity of the right wheel joint

$w_l$ : angular velocity of the left wheel joint

$w_s$: angular velocity of the steering joint

Cartesian space parameters:

In [6]:
(vx, vy, vz) = symbols('v_x v_y v_z')
xv = Matrix([vx, vy, vz])

$v_x$ : x axes velocity

$v_y$ : y axes velocity

$v_z$ : z axes velocity

# Dynamics of diff drive vehicle

In [7]:
diff_drive = Matrix([
    [r/2, r/2],
    [0, 0],
    [r/W, -r/W]
])
Eq(xv, MatMul(diff_drive, Matrix(uv[0:2]), evaluate=False), evaluate=False)

        ⎡r   r ⎤      
        ⎢─   ─ ⎥      
⎡vₓ ⎤   ⎢2   2 ⎥      
⎢   ⎥   ⎢      ⎥ ⎡wᵣ ⎤
⎢v_y⎥ = ⎢0   0 ⎥⋅⎢   ⎥
⎢   ⎥   ⎢      ⎥ ⎣w_l⎦
⎣v_z⎦   ⎢r  -r ⎥      
        ⎢─  ───⎥      
        ⎣W   W ⎦      

x axes velocity is generated from the combination of left and right wheel velocities.

z axes velocity is generated from the difference of left and right wheel velocities.

We can natually understand that we cannot directly obtain y axes velocity from the mechanism.

# Dynamics of offset diff drive vehicle

Here, we apply offset to the diff drive vehicle described above.

We first, create offset transformation matrix:

In [8]:
offset = Matrix([
    [1, 0, 0],
    [0, 1, s],
    [0, 0, 1]
])
offset

⎡1  0  0⎤
⎢       ⎥
⎢0  1  s⎥
⎢       ⎥
⎣0  0  1⎦

Apply offset to the jacobian matrix:

In [9]:
offset_diff_drive = offset * diff_drive
Eq(offset_diff_drive, MatMul(offset, diff_drive, evaluate=False), evaluate=False)

⎡ r     r  ⎤                     
⎢ ─     ─  ⎥             ⎡r   r ⎤
⎢ 2     2  ⎥             ⎢─   ─ ⎥
⎢          ⎥   ⎡1  0  0⎤ ⎢2   2 ⎥
⎢r⋅s  -r⋅s ⎥   ⎢       ⎥ ⎢      ⎥
⎢───  ─────⎥ = ⎢0  1  s⎥⋅⎢0   0 ⎥
⎢ W     W  ⎥   ⎢       ⎥ ⎢      ⎥
⎢          ⎥   ⎣0  0  1⎦ ⎢r  -r ⎥
⎢ r    -r  ⎥             ⎢─  ───⎥
⎢ ─    ─── ⎥             ⎣W   W ⎦
⎣ W     W  ⎦                     

Now, the equation will be as below:

In [10]:
Eq(xv, MatMul(offset_diff_drive, Matrix(uv[0:2]), evaluate=False), evaluate=False)

        ⎡ r     r  ⎤      
        ⎢ ─     ─  ⎥      
        ⎢ 2     2  ⎥      
⎡vₓ ⎤   ⎢          ⎥      
⎢   ⎥   ⎢r⋅s  -r⋅s ⎥ ⎡wᵣ ⎤
⎢v_y⎥ = ⎢───  ─────⎥⋅⎢   ⎥
⎢   ⎥   ⎢ W     W  ⎥ ⎣w_l⎦
⎣v_z⎦   ⎢          ⎥      
        ⎢ r    -r  ⎥      
        ⎢ ─    ─── ⎥      
        ⎣ W     W  ⎦      

What is interesting here, is we get y axes velocity by just adding the offset!

By calculating the inverse matrix, we can realize omnidirectional movement from this mechanism:

In [11]:
inv_offset_diff_drive = Matrix(offset_diff_drive[0:2, :]).inv()
Eq(Matrix(uv[0:2]), MatMul(inv_offset_diff_drive, Matrix(xv[0:2]), evaluate=False), evaluate=False)

        ⎡1    W  ⎤      
        ⎢─  ─────⎥      
⎡wᵣ ⎤   ⎢r  2⋅r⋅s⎥ ⎡vₓ ⎤
⎢   ⎥ = ⎢        ⎥⋅⎢   ⎥
⎣w_l⎦   ⎢1   -W  ⎥ ⎣v_y⎦
        ⎢─  ─────⎥      
        ⎣r  2⋅r⋅s⎦      

Unfortunately z axes velocity is defined by holonomic constraints and not controllable by us. But we will add a steering joint to our vehicle to realize complete omnidirectional movement.

# Dynamics of offset diff drive vehicle with a steering joint

Here, we will add a steering joint to our vehicle to get controllable z axes velocity:

In [12]:
rotate = Matrix([
    [cos(theta), -sin(theta), 0],
    [sin(theta), cos(theta), 0],
    [0, 0, 1]
])
rotate

⎡cos(\theta_s)  -sin(\theta_s)  0⎤
⎢                                ⎥
⎢sin(\theta_s)  cos(\theta_s)   0⎥
⎢                                ⎥
⎣      0              0         1⎦

By applying the rotation, jacobian matrix will be as follows:

In [13]:
abs_offset_diff_drive = rotate * offset_diff_drive
Eq(abs_offset_diff_drive, MatMul(rotate, offset_diff_drive, evaluate=False), evaluate=False)

⎡r⋅cos(\theta_s)   r⋅s⋅sin(\theta_s)  r⋅cos(\theta_s)   r⋅s⋅sin(\theta_s)⎤    
⎢─────────────── - ─────────────────  ─────────────── + ─────────────────⎥    
⎢       2                  W                 2                  W        ⎥    
⎢                                                                        ⎥   ⎡
⎢r⋅sin(\theta_s)   r⋅s⋅cos(\theta_s)  r⋅sin(\theta_s)   r⋅s⋅cos(\theta_s)⎥   ⎢
⎢─────────────── + ─────────────────  ─────────────── - ─────────────────⎥ = ⎢
⎢       2                  W                 2                  W        ⎥   ⎢
⎢                                                                        ⎥   ⎣
⎢                 r                                   -r                 ⎥    
⎢                 ─                                   ───                ⎥    
⎣                 W                                    W                 ⎦    

                                  ⎡ r     r  ⎤
                                  ⎢ ─     ─  ⎥
                                  ⎢ 

Final jacobian matrix is as follows:

In [14]:
fwd_dynamics = BlockMatrix([abs_offset_diff_drive, Matrix([0, 0, -1])]).as_explicit()
Eq(xv,MatMul(fwd_dynamics, uv, evaluate=False), evaluate=False)

        ⎡r⋅cos(\theta_s)   r⋅s⋅sin(\theta_s)  r⋅cos(\theta_s)   r⋅s⋅sin(\theta
        ⎢─────────────── - ─────────────────  ─────────────── + ──────────────
        ⎢       2                  W                 2                  W     
⎡vₓ ⎤   ⎢                                                                     
⎢   ⎥   ⎢r⋅sin(\theta_s)   r⋅s⋅cos(\theta_s)  r⋅sin(\theta_s)   r⋅s⋅cos(\theta
⎢v_y⎥ = ⎢─────────────── + ─────────────────  ─────────────── - ──────────────
⎢   ⎥   ⎢       2                  W                 2                  W     
⎣v_z⎦   ⎢                                                                     
        ⎢                 r                                   -r              
        ⎢                 ─                                   ───             
        ⎣                 W                                    W              

_s)    ⎤      
───  0 ⎥      
       ⎥      
       ⎥ ⎡wᵣ ⎤
_s)    ⎥ ⎢   ⎥
───  0 ⎥⋅⎢w_l⎥
       ⎥ ⎢   ⎥
       ⎥ ⎣w_s⎦
       ⎥  

Inverse jacobian matrix will let us know how to calculate each joint velocity to realize omnidirectional movement:

In [15]:
inv_dynamics = fwd_dynamics.inv().simplify()
Eq(uv, MatMul(inv_dynamics, xv, evaluate=False), evaluate=False)

        ⎡  W⋅sin(\theta_s)                     W⋅cos(\theta_s)                
        ⎢- ─────────────── + s⋅cos(\theta_s)   ─────────────── + s⋅sin(\theta_
        ⎢         2                                   2                       
        ⎢───────────────────────────────────   ───────────────────────────────
        ⎢                r⋅s                                  r⋅s             
⎡wᵣ ⎤   ⎢                                                                     
⎢   ⎥   ⎢ W⋅sin(\theta_s)                       W⋅cos(\theta_s)               
⎢w_l⎥ = ⎢ ─────────────── + s⋅cos(\theta_s)   - ─────────────── + s⋅sin(\theta
⎢   ⎥   ⎢        2                                     2                      
⎣w_s⎦   ⎢ ─────────────────────────────────   ────────────────────────────────
        ⎢                r⋅s                                  r⋅s             
        ⎢                                                                     
        ⎢          -sin(\theta_s)                   